In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
! pip install transformers
! pip install spacy
! python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 80.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import Matcher
import re
import os
import csv
import pickle
from collections import defaultdict
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [ ]:
# WITH PRE-PROCESSING STEP WHICH EXTRACTS TWEETS FIRST

# ------------------
# 1. SETUP & CONFIGURATION
# ------------------
# --- File Paths ---
C1_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/corpora/c1-left.csv"
C2_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/corpora/c2-right.csv"

#--UPDATE--
# --- read csv ---
# TARGETS_CSV_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/targets/tg.csv" # <-- CHANGE THIS FILENAME
# TEXT_COLUMN = "content"
# OUTPUT_DIR = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET"

# --- read txt ---
TARGETS_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/targets/canal.txt" # <-- CHANGE THIS FILENAME
TEXT_COLUMN = "content"
OUTPUT_DIR = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET"
# --- END OF UPDATE ---

# --- Intermediate & Final Files ---
MATCHES_CSV_PATH = os.path.join(OUTPUT_DIR, "found_matches.csv")
EMBEDDINGS_PKL_PATH = os.path.join(OUTPUT_DIR, "extracted_embeddings.pkl")
FINAL_RESULTS_PATH = os.path.join(OUTPUT_DIR, "oeuvre.xlsx")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Model & Device Setup ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Load models only once
TOKENIZER = AutoTokenizer.from_pretrained("almanach/camembertv2-base")
MODEL = AutoModel.from_pretrained("almanach/camembertv2-base").to(DEVICE).eval()
NLP = spacy.load("fr_core_news_sm", disable=["parser", "ner"])
torch.manual_seed(42)
np.random.seed(42)

# ------------------
# 2. HELPER FUNCTIONS
# ------------------
def extract_embedding(sentence, target):
    """Extracts the embedding for a target phrase within a sentence."""
    if not isinstance(sentence, str): return None
    try:
        match = re.search(re.escape(target), sentence)
        if not match: return None
        char_start, char_end = match.span()

        inputs = TOKENIZER(sentence, return_tensors="pt", truncation=True, max_length=512, return_offsets_mapping=True).to(DEVICE)
        offset_mapping = inputs.pop("offset_mapping").squeeze(0)

        token_indices = [i for i, (start, end) in enumerate(offset_mapping) if start >= char_start and end <= char_end and end > start]

        if not token_indices: return None

        with torch.no_grad():
            outputs = MODEL(**inputs).last_hidden_state.squeeze(0)

        return outputs[token_indices].mean(dim=0).cpu().numpy()
    except Exception:
        return None

def cosine_variance(matrix):
    """Computes mean pairwise cosine distance for a set of vectors."""
    if matrix.shape[0] < 2: return 0.0
    sim_matrix = cosine_similarity(matrix)
    upper = np.triu(sim_matrix, k=1)
    pairwise_distances = 1 - upper[upper != 0]
    return pairwise_distances.mean()

# ------------------
# 3. ANALYSIS STAGES
# ------------------
def run_stage_1_find_matches(corpus_path, corpus_id, targets_single, targets_multi, output_csv_path):
    """Processes a corpus once to find all matches and saves them to a CSV."""
    print(f"\n--- STAGE 1: Finding matches in {corpus_id} ---")
    df = pd.read_csv(corpus_path)
    texts = df[TEXT_COLUMN].dropna().tolist()

    matcher = Matcher(NLP.vocab)
    common_words = {"des", "de", "du", "la", "le", "les", "à"}

    for phrase in targets_multi:
        pattern = []
        for word in phrase.split():
            if word.lower() in common_words:
                pattern.append({"LOWER": word.lower()})
            else:
                pattern.append({"LEMMA": word})
        matcher.add(phrase, [pattern])

    with open(output_csv_path, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in tqdm(NLP.pipe(texts), total=len(texts), desc=f"Processing {corpus_id}"):
            # --- UPDATED & SIMPLIFIED MATCHING LOGIC ---

            # 1. Find and record all multi-word expressions first
            matches = matcher(doc)
            # Keep track of tokens that are already part of a multi-word match
            matched_tokens = set()
            for match_id, start, end in matches:
                phrase = NLP.vocab.strings[match_id]
                writer.writerow([phrase, corpus_id, doc.text, doc[start:end].text])
                for i in range(start, end):
                    matched_tokens.add(i)

            # 2. Find single words, ignoring any that were part of a multi-word match
            for i, token in enumerate(doc):
                if i in matched_tokens:
                    continue
                if token.lemma_ in targets_single:
                    writer.writerow([token.lemma_, corpus_id, doc.text, token.text])
            # --- END OF UPDATE ---

def run_stage_2_extract_embeddings(matches_csv_path, embeddings_pkl_path):
    """Reads matches from CSV and extracts embeddings, saving them to a pickle file."""
    print("\n--- STAGE 2: Extracting embeddings for all found matches ---")
    matches_df = pd.read_csv(matches_csv_path, names=["target_lemma", "corpus_id", "text", "matched_form"])

    embeddings = defaultdict(lambda: defaultdict(list))

    for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Extracting embeddings"):
        emb = extract_embedding(row['text'], row['matched_form'])
        if emb is not None:
            embeddings[row['target_lemma']][row['corpus_id']].append(emb)

    # A pickle-friendly way to save the nested defaultdict
    final_embeddings = {k: dict(v) for k, v in embeddings.items()}
    with open(embeddings_pkl_path, 'wb') as f:
        pickle.dump(final_embeddings, f)

def run_stage_3_calculate_results(embeddings_pkl_path, final_results_path, targets):
    """Loads embeddings and calculates final frequency, distance, and variance metrics."""
    print("\n--- STAGE 3: Calculating final distances and variance ---")
    with open(embeddings_pkl_path, 'rb') as f:
        embeddings = pickle.load(f)

    results = []
    for target in tqdm(targets, desc="Calculating metrics"):
        emb_c1 = np.array(embeddings.get(target, {}).get('c1', []))
        emb_c2 = np.array(embeddings.get(target, {}).get('c2', []))

        inter_dist, intra_c1, intra_c2 = None, None, None

        if emb_c1.size > 0 and emb_c2.size > 0:
            avg_c1 = emb_c1.mean(axis=0)
            avg_c2 = emb_c2.mean(axis=0)
            inter_dist = cosine_distances([avg_c1], [avg_c2])[0][0]
            intra_c1 = cosine_variance(emb_c1)
            intra_c2 = cosine_variance(emb_c2)

        results.append({
            "target": target,
            "inter_corpus_distance": inter_dist,
            "intra_variance_c1": intra_c1,
            "intra_variance_c2": intra_c2,
            "n_c1": len(emb_c1),
            "n_c2": len(emb_c2)
        })

    pd.DataFrame(results).to_excel(final_results_path, index=False)

# ------------------
# 4. MAIN EXECUTION
# ------------------

if __name__ == '__main__':
  # --- THIS SECTION IS UPDATED ---
    # Load targets from the specified TXT file
    print(f"Loading targets from: {TARGETS_PATH}")
    with open(TARGETS_PATH, "r", encoding="utf-8") as f:
        # Read each line, strip whitespace, convert to lowercase, and filter out empty lines
        targets = [line.strip().lower() for line in f if line.strip()]

    # Ensure targets are unique
    targets = sorted(list(set(targets)))
    print(f"Found {len(targets)} unique targets.")
    # --- END OF UPDATE ---

    # --- for csv ---
    # Load targets from the 'target' column of the specified CSV file
    #print(f"Loading targets from: {TARGETS_CSV_PATH}")
    #targets_df = pd.read_csv(TARGETS_CSV_PATH)

    # Get unique, non-empty targets from the 'target' column
    #targets = targets_df['target'].dropna().unique().tolist()
    #print(f"Found {len(targets)} unique targets.")
    # --- END OF UPDATE ---

    targets_single = {t for t in targets if " " not in t}
    targets_multi = [t for t in targets if " " in t]

    # --- Execute Stage 1: Find Matches ---
    if not os.path.exists(MATCHES_CSV_PATH):
        # Write header for new CSV
        with open(MATCHES_CSV_PATH, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["target_lemma", "corpus_id", "text", "matched_form"])
        # Run for each corpus
        run_stage_1_find_matches(C1_PATH, 'c1', targets_single, targets_multi, MATCHES_CSV_PATH)
        run_stage_1_find_matches(C2_PATH, 'c2', targets_single, targets_multi, MATCHES_CSV_PATH)
        print("✅ Stage 1 complete. All matches saved to 'found_matches.csv'.")
    else:
        print("⏭️ Stage 1 skipped: 'found_matches.csv' already exists.")

    # --- Execute Stage 2: Extract Embeddings ---
    if not os.path.exists(EMBEDDINGS_PKL_PATH):
        run_stage_2_extract_embeddings(MATCHES_CSV_PATH, EMBEDDINGS_PKL_PATH)
        print("✅ Stage 2 complete. All embeddings saved to 'extracted_embeddings.pkl'.")
    else:
        print("⏭️ Stage 2 skipped: 'extracted_embeddings.pkl' already exists.")

    # --- Execute Stage 3: Calculate Final Results ---
    run_stage_3_calculate_results(EMBEDDINGS_PKL_PATH, FINAL_RESULTS_PATH, targets)
    print(f"🎉 Analysis complete! Final results saved to '{FINAL_RESULTS_PATH}'.")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

...

✅ Stage 1 complete. All matches saved to 'found_matches.csv'.

--- STAGE 2: Extracting embeddings for all found matches ---


Extracting embeddings: 100%|██████████| 4652/4652 [01:04<00:00, 72.47it/s]


✅ Stage 2 complete. All embeddings saved to 'extracted_embeddings.pkl'.

--- STAGE 3: Calculating final distances and variance ---


Calculating metrics: 100%|██████████| 40/40 [00:00<00:00, 479.61it/s]


🎉 Analysis complete! Final results saved to '/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET/oeuvre.xlsx'.


In [ ]:
#ONLY FOR DROITS DES FEMMES

# ------------------
# 1. SETUP & CONFIGURATION
# ------------------
# --- File Paths ---
C1_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/corpora/c1-left.csv"
C2_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/corpora/c2-right.csv"

#--UPDATE--
# --- read csv ---
# TARGETS_CSV_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/targets/tg.csv" # <-- CHANGE THIS FILENAME
# TEXT_COLUMN = "content"
# OUTPUT_DIR = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET"

# --- read txt ---
TARGETS_PATH = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/targets/canal.txt" # <-- CHANGE THIS FILENAME
TEXT_COLUMN = "content"
OUTPUT_DIR = "/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET"
# --- END OF UPDATE ---

# --- Intermediate & Final Files ---
MATCHES_CSV_PATH = os.path.join(OUTPUT_DIR, "found_matches.csv")
EMBEDDINGS_PKL_PATH = os.path.join(OUTPUT_DIR, "extracted_embeddings.pkl")
FINAL_RESULTS_PATH = os.path.join(OUTPUT_DIR, "oeuvre.xlsx")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Model & Device Setup ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Load models only once
TOKENIZER = AutoTokenizer.from_pretrained("almanach/camembertv2-base")
MODEL = AutoModel.from_pretrained("almanach/camembertv2-base").to(DEVICE).eval()
NLP = spacy.load("fr_core_news_sm", disable=["parser", "ner"])
torch.manual_seed(42)
np.random.seed(42)

# ------------------
# 2. HELPER FUNCTIONS
# ------------------
def extract_embedding(sentence, target):
    """Extracts the embedding for a target phrase within a sentence."""
    if not isinstance(sentence, str): return None
    try:
        match = re.search(re.escape(target), sentence)
        if not match: return None
        char_start, char_end = match.span()

        inputs = TOKENIZER(sentence, return_tensors="pt", truncation=True, max_length=512, return_offsets_mapping=True).to(DEVICE)
        offset_mapping = inputs.pop("offset_mapping").squeeze(0)

        token_indices = [i for i, (start, end) in enumerate(offset_mapping) if start >= char_start and end <= char_end and end > start]

        if not token_indices: return None

        with torch.no_grad():
            outputs = MODEL(**inputs).last_hidden_state.squeeze(0)

        return outputs[token_indices].mean(dim=0).cpu().numpy()
    except Exception:
        return None

def cosine_variance(matrix):
    """Computes mean pairwise cosine distance for a set of vectors."""
    if matrix.shape[0] < 2: return 0.0
    sim_matrix = cosine_similarity(matrix)
    upper = np.triu(sim_matrix, k=1)
    pairwise_distances = 1 - upper[upper != 0]
    return pairwise_distances.mean()

# ------------------
# 3. ANALYSIS STAGES
# ------------------
def run_stage_1_find_matches(corpus_path, corpus_id, targets_single, targets_multi, output_csv_path):
    """Processes a corpus once to find all matches and saves them to a CSV."""
    print(f"\n--- STAGE 1: Finding matches in {corpus_id} ---")
    df = pd.read_csv(corpus_path)
    texts = df[TEXT_COLUMN].dropna().tolist()

     # --- UPDATED MATCHER LOGIC ---
    matcher = Matcher(NLP.vocab)
    common_words = {"des", "de", "du", "la", "le", "les", "à"}

    for phrase in targets_multi:
        # Process the target phrase itself with spaCy to get correct lemmas
        phrase_doc = NLP(phrase)
        pattern = []
        for token in phrase_doc:
            if token.lower_ in common_words:
                # For common words, match the exact lowercase text
                pattern.append({"LOWER": token.lower_})
            else:
                # For all other words, match their base lemma
                pattern.append({"LEMMA": token.lemma_})
        matcher.add(phrase, [pattern])
    # --- END OF UPDATE ---

    with open(output_csv_path, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in tqdm(NLP.pipe(texts), total=len(texts), desc=f"Processing {corpus_id}"):
            # --- UPDATED & SIMPLIFIED MATCHING LOGIC ---

            # 1. Find and record all multi-word expressions first
            matches = matcher(doc)
            # Keep track of tokens that are already part of a multi-word match
            matched_tokens = set()
            for match_id, start, end in matches:
                phrase = NLP.vocab.strings[match_id]
                writer.writerow([phrase, corpus_id, doc.text, doc[start:end].text])
                for i in range(start, end):
                    matched_tokens.add(i)

            # 2. Find single words, ignoring any that were part of a multi-word match
            for i, token in enumerate(doc):
                if i in matched_tokens:
                    continue
                if token.lemma_ in targets_single:
                    writer.writerow([token.lemma_, corpus_id, doc.text, token.text])
            # --- END OF UPDATE ---

def run_stage_2_extract_embeddings(matches_csv_path, embeddings_pkl_path):
    """Reads matches from CSV and extracts embeddings, saving them to a pickle file."""
    print("\n--- STAGE 2: Extracting embeddings for all found matches ---")
    matches_df = pd.read_csv(matches_csv_path, names=["target_lemma", "corpus_id", "text", "matched_form"])

    embeddings = defaultdict(lambda: defaultdict(list))

    for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Extracting embeddings"):
        emb = extract_embedding(row['text'], row['matched_form'])
        if emb is not None:
            embeddings[row['target_lemma']][row['corpus_id']].append(emb)

    # A pickle-friendly way to save the nested defaultdict
    final_embeddings = {k: dict(v) for k, v in embeddings.items()}
    with open(embeddings_pkl_path, 'wb') as f:
        pickle.dump(final_embeddings, f)

def run_stage_3_calculate_results(embeddings_pkl_path, final_results_path, targets):
    """Loads embeddings and calculates final frequency, distance, and variance metrics."""
    print("\n--- STAGE 3: Calculating final distances and variance ---")
    with open(embeddings_pkl_path, 'rb') as f:
        embeddings = pickle.load(f)

    results = []
    for target in tqdm(targets, desc="Calculating metrics"):
        emb_c1 = np.array(embeddings.get(target, {}).get('c1', []))
        emb_c2 = np.array(embeddings.get(target, {}).get('c2', []))

        inter_dist, intra_c1, intra_c2 = None, None, None

        if emb_c1.size > 0 and emb_c2.size > 0:
            avg_c1 = emb_c1.mean(axis=0)
            avg_c2 = emb_c2.mean(axis=0)
            inter_dist = cosine_distances([avg_c1], [avg_c2])[0][0]
            intra_c1 = cosine_variance(emb_c1)
            intra_c2 = cosine_variance(emb_c2)

        results.append({
            "target": target,
            "inter_corpus_distance": inter_dist,
            "intra_variance_c1": intra_c1,
            "intra_variance_c2": intra_c2,
            "n_c1": len(emb_c1),
            "n_c2": len(emb_c2)
        })

    pd.DataFrame(results).to_excel(final_results_path, index=False)

# ------------------
# 4. MAIN EXECUTION
# ------------------

if __name__ == '__main__':
  # --- THIS SECTION IS UPDATED ---
    # Load targets from the specified TXT file
    print(f"Loading targets from: {TARGETS_PATH}")
    with open(TARGETS_PATH, "r", encoding="utf-8") as f:
        # Read each line, strip whitespace, convert to lowercase, and filter out empty lines
        targets = [line.strip().lower() for line in f if line.strip()]

    # Ensure targets are unique
    targets = sorted(list(set(targets)))
    print(f"Found {len(targets)} unique targets.")
    # --- END OF UPDATE ---

    # --- for csv ---
    # Load targets from the 'target' column of the specified CSV file
    #print(f"Loading targets from: {TARGETS_CSV_PATH}")
    #targets_df = pd.read_csv(TARGETS_CSV_PATH)

    # Get unique, non-empty targets from the 'target' column
    #targets = targets_df['target'].dropna().unique().tolist()
    #print(f"Found {len(targets)} unique targets.")
    # --- END OF UPDATE ---

    targets_single = {t for t in targets if " " not in t}
    targets_multi = [t for t in targets if " " in t]

    # --- Execute Stage 1: Find Matches ---
    if not os.path.exists(MATCHES_CSV_PATH):
        # Write header for new CSV
        with open(MATCHES_CSV_PATH, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["target_lemma", "corpus_id", "text", "matched_form"])
        # Run for each corpus
        run_stage_1_find_matches(C1_PATH, 'c1', targets_single, targets_multi, MATCHES_CSV_PATH)
        run_stage_1_find_matches(C2_PATH, 'c2', targets_single, targets_multi, MATCHES_CSV_PATH)
        print("✅ Stage 1 complete. All matches saved to 'found_matches.csv'.")
    else:
        print("⏭️ Stage 1 skipped: 'found_matches.csv' already exists.")

    # --- Execute Stage 2: Extract Embeddings ---
    if not os.path.exists(EMBEDDINGS_PKL_PATH):
        run_stage_2_extract_embeddings(MATCHES_CSV_PATH, EMBEDDINGS_PKL_PATH)
        print("✅ Stage 2 complete. All embeddings saved to 'extracted_embeddings.pkl'.")
    else:
        print("⏭️ Stage 2 skipped: 'extracted_embeddings.pkl' already exists.")

    # --- Execute Stage 3: Calculate Final Results ---
    run_stage_3_calculate_results(EMBEDDINGS_PKL_PATH, FINAL_RESULTS_PATH, targets)
    print(f"🎉 Analysis complete! Final results saved to '{FINAL_RESULTS_PATH}'.")

Using device: cuda


Some weights of RobertaModel were not initialized from the model checkpoint at almanach/camembertv2-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading targets from: /content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/targets/canal.txt
Found 21 unique targets.

--- STAGE 1: Finding matches in c1 ---


Processing c1: 100%|██████████| 165187/165187 [04:55<00:00, 558.74it/s]



--- STAGE 1: Finding matches in c2 ---


...

✅ Stage 1 complete. All matches saved to 'found_matches.csv'.

--- STAGE 2: Extracting embeddings for all found matches ---


Extracting embeddings: 100%|██████████| 5070/5070 [01:05<00:00, 77.42it/s]


✅ Stage 2 complete. All embeddings saved to 'extracted_embeddings.pkl'.

--- STAGE 3: Calculating final distances and variance ---


Calculating metrics: 100%|██████████| 21/21 [00:00<00:00, 277.27it/s]


🎉 Analysis complete! Final results saved to '/content/drive/MyDrive/_thesis_dogwhistles/Compare-embeddings/results/DATASET/oeuvre.xlsx'.
